In [1]:
// Starting Spark Job
val sqlContext = new org.apache.spark.sql.SQLContext(sc)
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

import org.apache.spark.sql.types._

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.24:4040
SparkContext available as 'sc' (version = 2.4.7, master = local[*], app id = local-1612838809760)
SparkSession available as 'spark'


sqlContext: org.apache.spark.sql.SQLContext = org.apache.spark.sql.SQLContext@7a665db0
import org.apache.spark.sql.types._


In [2]:
val customSchema = StructType(Array(
    StructField("STN---", IntegerType, false),
    StructField("WBAN", IntegerType, false),
    StructField("YEARMODA", DateType, false),
    StructField("TEMP", DoubleType, true),
    StructField("DEWP", DoubleType, true),
    StructField("SLP", DoubleType, true),
    StructField("STP", DoubleType, true),
    StructField("VISIB", DoubleType, true),
    StructField("WDSP", DoubleType, true),
    StructField("MXSPD", DoubleType, true),
    StructField("GUST", DoubleType, true),
    StructField("MAX", StringType, true),
    StructField("MIN", StringType, true),
    StructField("PRCP", StringType, true),
    StructField("SNDP", DoubleType, true),
    StructField("FRSHTT", StringType, false))
)

customSchema: org.apache.spark.sql.types.StructType = StructType(StructField(STN---,IntegerType,false), StructField(WBAN,IntegerType,false), StructField(YEARMODA,DateType,false), StructField(TEMP,DoubleType,true), StructField(DEWP,DoubleType,true), StructField(SLP,DoubleType,true), StructField(STP,DoubleType,true), StructField(VISIB,DoubleType,true), StructField(WDSP,DoubleType,true), StructField(MXSPD,DoubleType,true), StructField(GUST,DoubleType,true), StructField(MAX,StringType,true), StructField(MIN,StringType,true), StructField(PRCP,StringType,true), StructField(SNDP,DoubleType,true), StructField(FRSHTT,StringType,false))


In [3]:
// Load files to DataFrame
val weather = spark.read
                .format("csv")
                .option("header", true)
                .option("delimiter", ",")
                .option("dateFormat", "yyyyMMdd")
                .schema(customSchema)
                .load("data/2019")
                .select("STN---", "YEARMODA", "TEMP", "WDSP", "FRSHTT")
                .withColumnRenamed("STN---", "STN_NO")

val stationList = spark.read
                .option("header", true)
                .csv("stationlist.csv")

val countryList = spark.read
                .option("header", true)
                .csv("countrylist.csv")

weather: org.apache.spark.sql.DataFrame = [STN_NO: int, YEARMODA: date ... 3 more fields]
stationList: org.apache.spark.sql.DataFrame = [STN_NO: string, COUNTRY_ABBR: string]
countryList: org.apache.spark.sql.DataFrame = [COUNTRY_ABBR: string, COUNTRY_FULL: string]


In [4]:
// Show schemas and sample data
weather.printSchema()
// stationList.printSchema()
// countryList.printSchema()
// weather.show(2, false)
// stationList.show(2, false)
// countryList.show(2, false)


root
 |-- STN_NO: integer (nullable = true)
 |-- YEARMODA: date (nullable = true)
 |-- TEMP: double (nullable = true)
 |-- WDSP: double (nullable = true)
 |-- FRSHTT: string (nullable = true)



In [5]:
// Prepare station - country DF
val stationCountry = stationList.join(countryList, stationList("COUNTRY_ABBR") === countryList("COUNTRY_ABBR"))
            .select("STN_NO", "COUNTRY_FULL")
            .cache()

stationCountry: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [STN_NO: string, COUNTRY_FULL: string]


In [6]:
/* 

Q1: Which country had the hottest average mean temperature over the year? 

*/

val stationTempDF = weather.select("STN_NO", "TEMP")
                            .cache()

val countryTempDF = stationTempDF.join(stationCountry, stationTempDF("STN_NO") === stationCountry("STN_NO"))
                                    .cache()

val maxMeanTemp = countryTempDF.select("COUNTRY_FULL", "TEMP")
        .filter($"TEMP" < 9999.9)
        .groupBy("COUNTRY_FULL")
        .agg(avg("TEMP").as("avg_temp"))
        .orderBy(desc("avg_temp"))
        .limit(1)
        .cache()
        .show


+------------+-----------------+
|COUNTRY_FULL|         avg_temp|
+------------+-----------------+
|    DJIBOUTI|90.06114457831325|
+------------+-----------------+



stationTempDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [STN_NO: int, TEMP: double]
countryTempDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [STN_NO: int, TEMP: double ... 2 more fields]
maxMeanTemp: Unit = ()


In [7]:
/* 

Q2: Which country had the most consecutive days of tornadoes/funnel cloud formations?

*/


val stationFRSHTTDF = weather.select("STN_NO", "YEARMODA", "FRSHTT")
                                .cache()

val countryTempDF = stationFRSHTTDF.join(stationCountry, stationFRSHTTDF("STN_NO") === stationCountry("STN_NO"))
                                    .cache()

countryTempDF.createOrReplaceTempView("countryTemp")

spark.sql("""WITH country_date_of_year AS (
                SELECT DISTINCT COUNTRY_FULL, 
                    YEARMODA, 
                    DAYOFYEAR(YEARMODA) AS DATEOFYEAR 
                FROM countryTemp
                WHERE FRSHTT % 10 = 1
                AND COUNTRY_FULL IS NOT NULL
                ORDER BY COUNTRY_FULL, YEARMODA 
            ), country_group AS (
                SELECT COUNTRY_FULL,
                    DATEOFYEAR - ROW_NUMBER() OVER (PARTITION BY COUNTRY_FULL ORDER BY DATEOFYEAR) AS grp
                FROM country_date_of_year
            ) , count_group AS (
                SELECT COUNTRY_FULL, grp, COUNT(*) AS group_count
                FROM country_group
                GROUP BY COUNTRY_FULL, grp
            ) SELECT DISTINCT COUNTRY_FULL
                FROM count_group 
                WHERE group_count IN (SELECT max(group_count) FROM count_group)
                """).show

+--------------+
|  COUNTRY_FULL|
+--------------+
|         JAPAN|
|        CANADA|
|CAYMAN ISLANDS|
|         INDIA|
|         ITALY|
| UNITED STATES|
|         GHANA|
+--------------+



stationFRSHTTDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [STN_NO: int, YEARMODA: date ... 1 more field]
countryTempDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [STN_NO: int, YEARMODA: date ... 3 more fields]


In [8]:
/* 

Q3: Which country had the second highest average mean wind speed over the year?

*/

val stationWDSPDF = weather.select("STN_NO", "WDSP")
                            .cache()

val countryWDSPDF = stationWDSPDF.join(stationCountry, stationWDSPDF("STN_NO") === stationCountry("STN_NO"))
                                    .cache()

val maxMeanTemp = countryWDSPDF.select("COUNTRY_FULL", "WDSP")
        .filter($"WDSP" < 999.9)
        .groupBy("COUNTRY_FULL")
        .agg(avg("WDSP").as("avg_wdsp"))
        .orderBy(desc("avg_wdsp"))
        .limit(2)
        .orderBy(asc("avg_wdsp"))
        .limit(1)
        .cache()
        .show


+------------+------------------+
|COUNTRY_FULL|          avg_wdsp|
+------------+------------------+
|       ARUBA|15.975683060109283|
+------------+------------------+



stationWDSPDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [STN_NO: int, WDSP: double]
countryWDSPDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [STN_NO: int, WDSP: double ... 2 more fields]
maxMeanTemp: Unit = ()
